# Basic QUBO HUBO solver

In [27]:
##### Initial setting #####
# x1 = q0 + 2q1 + 4q2 + --- + 2^(n-1)q(n-1)
# x2 = qn + 2q(n+1) + 4q(n+2) + --- 2^(n-1)q(2n-1)
# x1x2 = c
# Least square problem
# QUBO = (x1x2 - c)^2 - c^2
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
x1 = int(17) #10111
x2 = int(19) #10133
c = x1 * x2
print("first prime number: ", x1)
print("second prime number: ", x2)
print("RSA number: ", c)

first prime number:  17
second prime number:  19
RSA number:  323


## Factorization problem to HUBO

In [28]:
qubits = 5
max_d = format(len(str(2 * qubits)), '02')
QM = np. zeros((2 * qubits, 2 * qubits))
Q = {}

# Linear term
for j in range(qubits):
    for i in range(qubits):
        po1 = i
        po2 = qubits + j
        val = pow(2, 2 * (i + j)) - 2 * c * pow(2, i + j)
        exec("Q.update({('q%s','q%s'):%s})" % (
            format(po1 + 1, max_d), format(po2 + 1, max_d), format(val)))

# Quadratic term
for k in range(qubits):
    for i in range(qubits - 1):
        for j in range(i + 1, qubits):
            # 2^(i+j+2k+1)aiajbk
            po1 = i
            po2 = j
            po3 = qubits + k
            val = pow(2, i + j + 2 * k + 1)
            exec("Q.update({('q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                         format(po3 + 1, max_d), format(val)))

# Cubic term
for k in range(qubits):
    for i in range(qubits - 1):
        for j in range(i + 1, qubits):
            # 2^(i+j+2k+1)akbibj
            po1 = k
            po2 = qubits + i
            po3 = qubits + j
            val = pow(2, i + j + 2 * k + 1)
            exec("Q.update({('q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                         format(po3 + 1, max_d), format(val)))

# Quartic term
for i2 in range(qubits - 1):
    for j2 in range(i2 + 1, qubits):
        for i1 in range(qubits - 1):
            for j1 in range(i1 + 1, qubits):
                po1 = i1
                po2 = j1
                po3 = qubits + i2
                po4 = qubits + j2
                val = pow(2, i1 + j1 + i2 + j2 + 2)
                exec("Q.update({('q%s','q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                                   format(po3 + 1, max_d), format(po4 + 1, max_d), format(val)))
                


## HUBO to QUBO and solving

In [35]:
from dimod import ExactSolver
from dwave.system import LeapHybridSampler

hybrid_sampler = LeapHybridSampler()  
sampler = DWaveSampler('')
sampler_auto = EmbeddingComposite(sampler)

print("Connected to sampler", sampler.solver.name)
print("Connected to hybrid_sampler", hybrid_sampler.solver.name)

# sampleset = dimod.ExactPolySolver().sample_hubo(Q)
# hubo = dimod.higherorder.BinaryPolynomial.from_hubo(Q)

# ising = dimod.higherorder.BinaryPolynomial.to_binary(hubo)
print(Q)

# qubo = dimod.make_quadratic_cqm(Q, dimod.BINARY)

# qubo = dimod.make_quadratic(Q, 6, dimod.BINARY)

# print(qubo)

sampleset = dimod.ExactPolySolver().sample_hubo(Q)

# sampleset = sampler.sample(qubo)

# sampleset = hybrid_sampler.sample(Q)


Connected to sampler Advantage_system4.1
Connected to hybrid_sampler hybrid_binary_quadratic_model_version2
{('q01', 'q06'): -645, ('q02', 'q06'): -1288, ('q03', 'q06'): -2568, ('q04', 'q06'): -5104, ('q05', 'q06'): -10080, ('q01', 'q07'): -1288, ('q02', 'q07'): -2568, ('q03', 'q07'): -5104, ('q04', 'q07'): -10080, ('q05', 'q07'): -19648, ('q01', 'q08'): -2568, ('q02', 'q08'): -5104, ('q03', 'q08'): -10080, ('q04', 'q08'): -19648, ('q05', 'q08'): -37248, ('q01', 'q09'): -5104, ('q02', 'q09'): -10080, ('q03', 'q09'): -19648, ('q04', 'q09'): -37248, ('q05', 'q09'): -66304, ('q01', 'q10'): -10080, ('q02', 'q10'): -19648, ('q03', 'q10'): -37248, ('q04', 'q10'): -66304, ('q05', 'q10'): -99840, ('q01', 'q02', 'q06'): 4, ('q01', 'q03', 'q06'): 8, ('q01', 'q04', 'q06'): 16, ('q01', 'q05', 'q06'): 32, ('q02', 'q03', 'q06'): 16, ('q02', 'q04', 'q06'): 32, ('q02', 'q05', 'q06'): 64, ('q03', 'q04', 'q06'): 64, ('q03', 'q05', 'q06'): 128, ('q04', 'q05', 'q06'): 256, ('q01', 'q02', 'q07'): 16, ('q01

In [34]:

# print(sampleset)

energy = 0
energies = sampleset.record.energy
energy0_nums = np. where(energies == - pow(c,2))[0]
x = np. zeros(2)
for idx in range(len(energy0_nums)):
    sol1 = sampleset.record[energy0_nums[idx]][0]
    for xk in range(2):
        x[xk] = 0
    lambda1 = 0
    for xk in range(2):
        for k in range(qubits):
            x[xk] = x[xk] + pow(2, k) * (sol1[xk * qubits + k])
    print(x)


     q01 q02 q03 q04 q05 q06 q07 q08 q09 q10    energy num_oc.
563    1   1   0   0   1   1   0   0   0   1 -104329.0       1
579    1   0   0   0   1   1   1   0   0   1 -104329.0       1
365    0   1   1   1   0   1   1   1   0   1 -104328.0       1
378    0   0   1   1   0   1   1   0   1   1 -104328.0       1
624    0   1   0   0   1   0   1   0   0   1 -104328.0       1
855    1   1   1   0   1   0   1   1   1   0 -104328.0       1
982    1   1   0   1   1   0   0   1   1   0 -104328.0       1
314    1   0   1   1   0   1   0   0   1   1 -104325.0       1
986    1   0   0   1   1   1   0   1   1   0 -104325.0       1
543    0   0   0   0   1   0   0   1   0   1 -104320.0       1
767    0   0   1   0   1   0   0   0   0   1 -104320.0       1
469    1   1   0   1   0   1   0   1   1   1 -104313.0       1
837    1   0   1   1   1   1   1   0   1   0 -104313.0       1
337    1   1   1   1   0   0   1   1   0   1 -104280.0       1
425    1   1   0   1   0   0   1   1   1   1 -104280.0 